In [17]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json
/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json
/kaggle/input/glove-twitter/glove.twitter.27B.200d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.25d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.50d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.100d.txt
/kaggle/input/glove-embeddings/glove.6B.200d.txt
/kaggle/input/glove-embeddings/glove.6B.50d.txt
/kaggle/input/glove-embeddings/glove.6B.300d.txt
/kaggle/input/glove-embeddings/glove.6B.100d.txt


In [18]:
import json
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

In [19]:
file = open("/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json")
json_data = [json.loads(line) for line in file]
json_data[0:5]

[{'is_sarcastic': 1,
  'headline': 'thirtysomething scientists unveil doomsday clock of hair loss',
  'article_link': 'https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205'},
 {'is_sarcastic': 0,
  'headline': 'dem rep. totally nails why congress is falling short on gender, racial equality',
  'article_link': 'https://www.huffingtonpost.com/entry/donna-edwards-inequality_us_57455f7fe4b055bb1170b207'},
 {'is_sarcastic': 0,
  'headline': 'eat your veggies: 9 deliciously different recipes',
  'article_link': 'https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html'},
 {'is_sarcastic': 1,
  'headline': 'inclement weather prevents liar from getting to work',
  'article_link': 'https://local.theonion.com/inclement-weather-prevents-liar-from-getting-to-work-1819576031'},
 {'is_sarcastic': 1,
  'headline': "mother comes pretty close to using word 'streaming' correctly",
  'article_link': 'https://www.theonion.com/mother-comes-pretty-

In [20]:
texts = []
labels = []
urls = []

for item in json_data:
    texts.append(item["headline"])
    labels.append(item["is_sarcastic"])
    urls.append(item["article_link"])
    
texts[0:10]

['thirtysomething scientists unveil doomsday clock of hair loss',
 'dem rep. totally nails why congress is falling short on gender, racial equality',
 'eat your veggies: 9 deliciously different recipes',
 'inclement weather prevents liar from getting to work',
 "mother comes pretty close to using word 'streaming' correctly",
 'my white inheritance',
 '5 ways to file your taxes with less stress',
 "richard branson's global-warming donation nearly as much as cost of failed balloon trips",
 'shadow government getting too large to meet in marriott conference room b',
 'lots of parents know this scenario']

In [21]:
labels[0:10]

[1, 0, 0, 1, 1, 0, 0, 1, 1, 0]

In [22]:
urls[0:5]

['https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205',
 'https://www.huffingtonpost.com/entry/donna-edwards-inequality_us_57455f7fe4b055bb1170b207',
 'https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html',
 'https://local.theonion.com/inclement-weather-prevents-liar-from-getting-to-work-1819576031',
 'https://www.theonion.com/mother-comes-pretty-close-to-using-word-streaming-cor-1819575546']

In [23]:
len(texts), len(labels), len(urls)

(28619, 28619, 28619)

In [24]:
df = pd.DataFrame()
df["text"] = texts
df["label"] = labels
df.head(n = 10).style.background_gradient(cmap = "autumn")

,text,label
0,thirtysomething scientists unveil doomsday clock of hair loss,1
1,"dem rep. totally nails why congress is falling short on gender, racial equality",0
2,eat your veggies: 9 deliciously different recipes,0
3,inclement weather prevents liar from getting to work,1
4,mother comes pretty close to using word 'streaming' correctly,1
5,my white inheritance,0
6,5 ways to file your taxes with less stress,0
7,richard branson's global-warming donation nearly as much as cost of failed balloon trips,1
8,shadow government getting too large to meet in marriott conference room b,1
9,lots of parents know this scenario,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    28619 non-null  object
 1   label   28619 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 447.3+ KB


In [26]:
df.duplicated().sum()

116

In [27]:
df.drop_duplicates(inplace = True)

In [28]:
df.groupby("label").count().style.background_gradient(cmap = "autumn")

,text
label,
0,14951
1,13552


In [33]:
df["text"] = df["text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
df["text"] = df["text"].str.replace('[^\w\s]','')
df["text"] = df["text"].str.replace('\d','')
sw = stopwords.words("english")
df["text"] = df["text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
# df["text"] = df["text"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

df.head(n = 10).style.background_gradient(cmap = "summer")

/tmp/ipykernel_28/1098870372.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace('[^\w\s]','')
/tmp/ipykernel_28/1098870372.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace('\d','')


,text,label
0,thirtysomething scientists unveil doomsday clock hair loss,1
1,dem rep totally nails congress falling short gender racial equality,0
2,eat veggies deliciously different recipes,0
3,inclement weather prevents liar getting work,1
4,mother comes pretty close using word streaming correctly,1
5,white inheritance,0
6,ways file taxes less stress,0
7,richard bransons globalwarming donation nearly much cost failed balloon trips,1
8,shadow government getting large meet marriott conference room b,1
9,lots parents know scenario,0


In [34]:
max_features = 20000
maxlen = 25
embedding_size = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['text']))
X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, maxlen = maxlen)
y = df['label']

In [35]:
EMBEDDING_FILE = '/kaggle/input/glove-embeddings/glove.6B.200d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if await self.run_code(code, result, async_=asy):


In [36]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 200)         4000000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        337920    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 40)                10280     
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense_1 (Dense)             (None, 20)                8

In [38]:
batch_size = 100
epochs = 5
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/5
229/229 [==============================] - 28s 91ms/step - loss: 0.6236 - accuracy: 0.6481 - val_loss: 0.4749 - val_accuracy: 0.7857
Epoch 2/5
229/229 [==============================] - 6s 26ms/step - loss: 0.4412 - accuracy: 0.8148 - val_loss: 0.4266 - val_accuracy: 0.8153
Epoch 3/5
229/229 [==============================] - 4s 17ms/step - loss: 0.2940 - accuracy: 0.8909 - val_loss: 0.3930 - val_accuracy: 0.8337
Epoch 4/5
229/229 [==============================] - 4s 20ms/step - loss: 0.1948 - accuracy: 0.9321 - val_loss: 0.4961 - val_accuracy: 0.8313
Epoch 5/5
229/229 [==============================] - 4s 16ms/step - loss: 0.1249 - accuracy: 0.9583 - val_loss: 0.6474 - val_accuracy: 0.8318
